In [ ]:
# Mars Anomaly Detection System
#This notebook analyzes Mars surface images using AI vision models to detect anomalies and generate professional reports.


In [ ]:
## 1. Configuration and Setup

In [ ]:
# Configuration settings – easily customizable
CONFIG = {
    'api_delay': 10,                         # Seconds between API calls
    'model_id': 'qwen/qwen2.5-vl-32b-instruct:free',
    'image_folder': 'Ground_images',
    'output_file': 'Mars_Anomaly_Report.pdf',
    'api_timeout': 30,                       # Timeout for API requests (seconds)
    'max_retries': 3,                        # Maximum retry attempts per request
    'supported_formats': ['.jpeg', '.jpg', '.png']  # Supported image formats
}

print("Configuration loaded successfully!")
print(f"Image folder: {CONFIG['image_folder']}")
print(f"Output file: {CONFIG['output_file']}")
print(f"Model: {CONFIG['model_id']}")


In [ ]:
# Install required packages
!pip install openai==0.28 python-dotenv fpdf pillow requests tqdm


In [ ]:
# Import necessary libraries
import os
import base64
import requests
import time
import re
import logging
from glob import glob
from dotenv import load_dotenv
from fpdf import FPDF
from PIL import Image
from tqdm import tqdm
from typing import List, Tuple, Optional

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('mars_analysis.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

print("Libraries imported successfully!")


In [ ]:
## 2. API Configuration and Authentication

In [ ]:
# Load environment variables
load_dotenv()

# Get API key with validation
api_key = os.getenv("OPENROUTER_API_KEY")
if not api_key:
    raise ValueError("OPENROUTER_API_KEY not found in environment variables. Please check your .env file.")

# API configuration
api_url = "https://openrouter.ai/api/v1/chat/completions"

print("✅ API key loaded successfully!")
print(f"API URL: {api_url}")
logger.info("API configuration completed")


In [ ]:
## 3. Utility Functions

In [ ]:
def validate_image_file(image_path: str) -> bool:
    """
    Validate if the image file exists and is in a supported format.
    """
    try:
        if not os.path.exists(image_path):
            logger.error(f"Image file not found: {image_path}")
            return False
        _, ext = os.path.splitext(image_path.lower())
        if ext not in CONFIG['supported_formats']:
            logger.error(f"Unsupported image format: {ext}")
            return False
        with Image.open(image_path) as img:
            img.verify()
        return True
    except Exception as e:
        logger.error(f"Error validating image {image_path}: {str(e)}")
        return False

def encode_image_to_base64(image_path: str) -> Optional[str]:
    """
    Encode image to base64 with error handling.
    """
    try:
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')
    except Exception as e:
        logger.error(f"Error encoding image {image_path}: {str(e)}")
        return None

def extract_gps_from_filename(image_path: str) -> str:
    """
    Extract GPS coordinates from filename with fallback handling.
    """
    try:
        filename = os.path.splitext(os.path.basename(image_path))[0]
        if re.search(r'\d+[._-]\d+', filename):
            return filename
        logger.warning(f"No GPS coordinates found in filename: {filename}")
        return f"Location_Unknown_{filename}"
    except Exception as e:
        logger.error(f"Error extracting GPS from filename: {str(e)}")
        return "Location_Error"

print("✅ Utility functions defined successfully!")


In [ ]:
## 4. Image Analysis Function

In [13]:
def load_prompt_template() -> str:
    """Load prompt template from a text file."""
    try:
        with open("prompt.txt", "r", encoding="utf-8") as f:
            return f.read()
    except FileNotFoundError:
        logger.error("prompt.txt file not found.")
        return ""

def analyze_image(image_path: str) -> str:
    """
    Analyze image using AI vision model with comprehensive error handling.
    """
    logger.info(f"Starting analysis of image: {image_path}")

    if not validate_image_file(image_path):
        return "Error: Invalid or corrupted image file"

    gps_coords = extract_gps_from_filename(image_path)
    base64_image = encode_image_to_base64(image_path)
    if not base64_image:
        return "Error: Failed to encode image to base64"

    # Load and format the prompt with GPS coordinates
    prompt_template = load_prompt_template()
    if not prompt_template:
        return "Error: Failed to load prompt template"
    
    formatted_prompt = prompt_template.format(gps_coords=gps_coords)

    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

    payload = {
        "model": CONFIG['model_id'],
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": formatted_prompt
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_image}"
                        }
                    }
                ]
            }
        ],
        "max_tokens": 500
    }

    for attempt in range(CONFIG['max_retries']):
        try:
            logger.info(f"API request attempt {attempt + 1}/{CONFIG['max_retries']}")
            response = requests.post(
                api_url,
                headers=headers,
                json=payload,
                timeout=CONFIG['api_timeout']
            )

            if response.status_code == 200:
                result = response.json()
                if 'choices' in result and result['choices']:
                    analysis = result['choices'][0]['message']['content']
                    logger.info(f"Successful analysis for {image_path}")
                    return analysis
                logger.error(f"Invalid response structure: {result}")
                return "Error: Invalid response from AI model"

            elif response.status_code == 429:
                wait_time = CONFIG['api_delay'] * (attempt + 1)
                logger.warning(f"Rate limit exceeded. Waiting {wait_time} seconds...")
                time.sleep(wait_time)
                continue

            else:
                logger.error(f"API request failed with status {response.status_code}: {response.text}")
                return f"Error: API request failed (Status: {response.status_code})"

        except requests.exceptions.Timeout:
            logger.error(f"Request timeout on attempt {attempt + 1}")
            if attempt == CONFIG['max_retries'] - 1:
                return "Error: Request timeout after multiple attempts"
            time.sleep(CONFIG['api_delay'])

        except requests.exceptions.RequestException as e:
            logger.error(f"Request exception on attempt {attempt + 1}: {str(e)}")
            if attempt == CONFIG['max_retries'] - 1:
                return f"Error: Network request failed - {str(e)}"
            time.sleep(CONFIG['api_delay'])

        except Exception as e:
            logger.error(f"Unexpected error during analysis: {str(e)}")
            return f"Error: Unexpected error during analysis - {str(e)}"

    return "Error: Maximum retry attempts exceeded"

print("✅ Enhanced image analysis function now loads prompt from prompt.txt!")


✅ Enhanced image analysis function now loads prompt from prompt.txt!


In [ ]:
## 5. Process Multiple Images

In [14]:
# === Cell 13: Image Processing Function (Fixed Imports) ===

import os
import time
from glob import glob
from typing import List, Tuple
from tqdm import tqdm

def process_images() -> List[Tuple[str, str, str]]:
    """
    Process all images in the configured folder.
    
    Returns:
        List[Tuple[str, str, str]]: List of (gps_coords, image_path, analysis) tuples
    """
    results = []

    # Check if image folder exists
    if not os.path.exists(CONFIG['image_folder']):
        logger.error(f"Image folder not found: {CONFIG['image_folder']}")
        print(f"❌ Error: Image folder '{CONFIG['image_folder']}' not found!")
        return results

    # Get all image files matching supported formats
    patterns = [f"{CONFIG['image_folder']}/*{ext}" for ext in CONFIG['supported_formats']]
    image_paths = []
    for pattern in patterns:
        image_paths.extend(glob(pattern))

    if not image_paths:
        logger.warning(f"No images found in {CONFIG['image_folder']}")
        print(f"⚠️ No images found in '{CONFIG['image_folder']}' folder!")
        return results

    print(f"📁 Found {len(image_paths)} images to process")
    logger.info(f"Processing {len(image_paths)} images")

    # Process images with progress tracking
    success, fail = 0, 0
    for img_path in tqdm(image_paths, desc="Analyzing images", unit="image"):
        gps = extract_gps_from_filename(img_path)
        try:
            analysis = analyze_image(img_path)
            results.append((gps, img_path, analysis))
            
            if analysis.startswith("Error:"):
                fail += 1
            else:
                success += 1
                
        except Exception as e:
            logger.error(f"Error processing {img_path}: {str(e)}")
            results.append((gps, img_path, f"Error: {str(e)}"))
            fail += 1

        # Delay between API calls (except for the last image)
        if img_path != image_paths[-1]:
            time.sleep(CONFIG['api_delay'])

    # Print processing summary
    print("\n📊 Processing Summary:")
    print(f"✅ Successful analyses: {success}")
    print(f"❌ Failed analyses: {fail}")
    print(f"📝 Total results: {len(results)}")
    logger.info(f"Completed processing. Success: {success}, Failed: {fail}")
    
    return results

print("✅ Image processing function defined!")


✅ Image processing function defined!


In [15]:
print("🚀 Starting image analysis...\n")
results = process_images()

if results:
    print(f"\n🎉 Analysis complete! Found {len(results)} results.")
else:
    print("\n⚠️ No results to process. Please check your image folder and try again.")


2025-07-29 14:50:45,281 - INFO - Processing 6 images


🚀 Starting image analysis...

📁 Found 6 images to process


Analyzing images:   0%|          | 0/6 [00:00<?, ?image/s]2025-07-29 14:50:45,285 - INFO - Starting analysis of image: Ground_images/-4.5892_137.4417.jpeg
2025-07-29 14:50:45,287 - INFO - API request attempt 1/3
2025-07-29 14:51:16,996 - INFO - Successful analysis for Ground_images/-4.5892_137.4417.jpeg
Analyzing images:  17%|█▋        | 1/6 [00:41<03:28, 41.71s/image]2025-07-29 14:51:26,999 - INFO - Starting analysis of image: Ground_images/48.8566_2.3522.jpeg
2025-07-29 14:51:27,002 - INFO - API request attempt 1/3
2025-07-29 14:51:47,699 - INFO - Successful analysis for Ground_images/48.8566_2.3522.jpeg
Analyzing images:  33%|███▎      | 2/6 [01:12<02:20, 35.24s/image]2025-07-29 14:51:57,702 - INFO - Starting analysis of image: Ground_images/22.1234_114.5678.jpeg
2025-07-29 14:51:57,706 - INFO - API request attempt 1/3
2025-07-29 14:52:28,162 - INFO - Successful analysis for Ground_images/22.1234_114.5678.jpeg
Analyzing images:  50%|█████     | 3/6 [01:52<01:52, 37.62s/image]2025-07


📊 Processing Summary:
✅ Successful analyses: 6
❌ Failed analyses: 0
📝 Total results: 6

🎉 Analysis complete! Found 6 results.


In [ ]:
## 6. PDF Report Generation 

In [16]:
def sanitize_text_for_pdf(text: str) -> str:
    """
    Replace Unicode characters that cause Latin-1 encoding issues.
    
    Args:
        text (str): Original text that may contain problematic Unicode characters
        
    Returns:
        str: Cleaned text safe for FPDF Latin-1 encoding
    """
    if not text:
        return ""
    
    try:
        # Replace smart quotes with regular quotes
        text = text.replace(''', "'")  # Left single quotation mark
        text = text.replace(''', "'")  # Right single quotation mark  
        text = text.replace('"', '"')  # Left double quotation mark
        text = text.replace('"', '"')  # Right double quotation mark
        
        # Replace dashes
        text = text.replace('—', '-')  # Em dash
        text = text.replace('–', '-')  # En dash
        
        # Replace other common problematic characters
        text = text.replace('…', '...')  # Horizontal ellipsis
        text = text.replace('®', '(R)')  # Registered trademark
        text = text.replace('©', '(C)')  # Copyright
        text = text.replace('™', '(TM)') # Trademark
        
        # Final safety net: encode to latin-1 with replacement
        text = text.encode('latin-1', errors='replace').decode('latin-1')
        
        return text
        
    except Exception as e:
        logger.error(f"Error sanitizing text: {str(e)}")
        # Last resort: force Latin-1 with question mark replacements
        return text.encode('latin-1', errors='replace').decode('latin-1')

print("Cleaned the text")


Cleaned the text


In [17]:
from fpdf import FPDF
import time
import re
from typing import List, Tuple

# Helper: Clean markdown formatting
def clean_markdown(text: str) -> str:
    """
    Remove markdown syntax for PDF display.
    """
    if not text:
        return "No analysis available"
    text = re.sub(r'#+\s*', '', text)
    text = re.sub(r'\*\*(.*?)\*\*', r'\1', text)
    text = re.sub(r'\[(.*?)\]\((.*?)\)', r'\1', text)
    text = re.sub(r'[`>]+', '', text)
    text = text.replace('---', '')
    return text.strip()

# Helper: Make text Latin-1 safe
def make_pdf_safe(text: str) -> str:
    return text.encode("latin-1", "ignore").decode("latin-1")

def create_enhanced_pdf_report(results: List[Tuple[str, str, str]]) -> bool:
    """
    Generate PDF with proper image pagination and bold labels.
    """
    if not results:
        logger.error("No results to create PDF report")
        return False

    try:
        logger.info(f"Creating PDF report: {CONFIG['output_file']}")
        
        class MarsReportPDF(FPDF):
            def header(self):
                # Only show title on page 1
                if self.page_no() == 1:
                    self.set_font('Arial', 'B', 16)
                    self.cell(0, 10, 'Mars Surface Anomaly Detection Report', 0, 1, 'C')
                    self.ln(5)
                else:
                    # Show TEAM SHUNYA on all other pages
                    self.set_font('Arial', 'B', 10)
                    self.cell(0, 10, 'TEAM SHUNYA', 0, 0, 'R')
                    self.ln(12)

            def footer(self):
                self.set_y(-15)
                self.set_font('Arial', 'I', 8)
                self.cell(0, 10, f'Page {self.page_no()}', 0, 0, 'C')

        pdf = MarsReportPDF()
        pdf.set_auto_page_break(auto=True, margin=15)

        # --- Title page (Page 1) ---
        pdf.add_page()
        pdf.set_font('Arial', 'B', 24)
        pdf.cell(0, 30, 'Mars Surface Analysis', 0, 1, 'C')
        pdf.set_font('Arial', 'B', 16)
        pdf.cell(0, 10, 'TEAM SHUNYA', 0, 1, 'C')
        pdf.ln(20)

        # Summary stats
        success = sum(1 for _, _, a in results if not a.startswith("Error:"))
        fail = len(results) - success
        pdf.set_font('Arial', 'B', 12)
        pdf.cell(0, 10, 'Summary:', 0, 1, 'L')
        pdf.set_font('Arial', '', 10)
        pdf.cell(0, 8, f'Successful Analyses: {success}', 0, 1, 'L')
        pdf.cell(0, 8, f'Failed Analyses: {fail}', 0, 1, 'L')

        # --- Detail pages ---
        processed = 0
        
        for gps_coords, img_path, analysis in results:
            lines = clean_markdown(analysis).split('\n')
            
            pdf.add_page()
            
            # GPS header
            pdf.set_font('Arial', 'B', 11)
            pdf.cell(0, 8, make_pdf_safe(f"GPS: {gps_coords}"), 0, 1, 'L')
            pdf.ln(4)

            # Process each line - BOLD ANY LABEL BEFORE COLON
            for line in lines:
                line = line.strip()
                if not line:
                    continue
                
                # Check if line contains a colon
                if ':' in line:
                    label, content = line.split(':', 1)
                    label = label.strip()
                    content = content.strip()
                    
                    # BOLD THE LABEL - put on separate line to ensure it works
                    pdf.set_font('Arial', 'B', 11)
                    pdf.cell(0, 7, make_pdf_safe(f"{label}:"), 0, 1, 'L')
                    
                    # Normal text for content on next line
                    if content:
                        pdf.set_font('Arial', '', 10)
                        pdf.multi_cell(0, 6, make_pdf_safe(content))
                    pdf.ln(3)  # Space after each section
                else:
                    # Regular paragraph text
                    pdf.set_font('Arial', '', 10)
                    pdf.multi_cell(0, 6, make_pdf_safe(line))
                    pdf.ln(2)

            # PROPER IMAGE PAGINATION - Check available space
            pdf.ln(5)
            try:
                if validate_image_file(img_path):
                    img_height = 100  # Estimated image height in mm
                    current_y = pdf.get_y()
                    page_height = 297  # A4 height in mm
                    bottom_margin = 15
                    
                    # Check if image fits on current page
                    if current_y + img_height > page_height - bottom_margin:
                        # Not enough space - create new page
                        pdf.add_page()
                        pdf.set_font('Arial', 'B', 10)
                        pdf.cell(0, 8, make_pdf_safe(f"Image for GPS: {gps_coords}"), 0, 1, 'L')
                        pdf.ln(3)
                    
                    # Add image centered
                    img_w = 140
                    page_w = 210
                    x_center = (page_w - img_w) / 2
                    pdf.image(img_path, x=x_center, y=pdf.get_y(), w=img_w)
                    
            except Exception as e:
                logger.warning(f"Image render error for {img_path}: {e}")
                # Add error message instead of image
                pdf.set_font('Arial', 'I', 9)
                pdf.cell(0, 6, '[Image could not be loaded]', 0, 1, 'L')
            
            processed += 1

        # Save PDF
        pdf.output(CONFIG['output_file'])
        logger.info(f"PDF created successfully with {processed} analyses")
        return True

    except Exception as e:
        logger.error(f"PDF creation failed: {e}")
        return False

print("Enchanced the text")


Enchanced the text


In [18]:
if results:
    print("📄 Generating PDF report...")
    success = create_enhanced_pdf_report(results)
    if success:
        print("\n🎉 PDF report generated successfully!")
        print(f"📁 File saved as: {CONFIG['output_file']}")
        if os.path.exists(CONFIG['output_file']):
            size = os.path.getsize(CONFIG['output_file'])
            print(f"📊 File size: {size:,} bytes ({size/1024/1024:.2f} MB)")
    else:
        print("\n❌ Failed to generate PDF report. Check the logs for details.")
else:
    print("\n⚠️ No results available to generate report.")


2025-07-29 14:54:20,031 - INFO - Creating PDF report: Mars_Anomaly_Report.pdf
2025-07-29 14:54:20,103 - INFO - PDF created successfully with 6 analyses


📄 Generating PDF report...

🎉 PDF report generated successfully!
📁 File saved as: Mars_Anomaly_Report.pdf
📊 File size: 1,190,658 bytes (1.14 MB)


In [ ]:
## 7. Summary and Next Steps

In [ ]:
print("\n" + "="*50)
print("📋 MARS ANOMALY DETECTION - FINAL SUMMARY")
print("="*50)

if results:
    succ = sum(1 for _, _, a in results if not a.startswith("Error:"))
    fail = len(results) - succ
    print(f"📊 Total Images Processed: {len(results)}")
    print(f"✅ Successful Analyses: {succ}")
    print(f"❌ Failed Analyses: {fail}")
    if succ:
        rate = (succ / len(results)) * 100
        print(f"📈 Success Rate: {rate:.1f}%")
    if os.path.exists(CONFIG['output_file']):
        print(f"📄 PDF Report: {CONFIG['output_file']} ✅")
    print(f"📝 Log File: mars_analysis.log")

else:
    print("⚠️  No images were processed. Please check:")
    print(f"   - Image folder exists: {CONFIG['image_folder']}")
    print(f"   - Images are in supported formats: {CONFIG['supported_formats']}")
    print("   - API key is correctly configured")
    print("   - Internet connection is working")

print("\n🚀 Analysis complete! Check the PDF report and log files for detailed results.")
logger.info("Mars anomaly detection process completed")
